In [1]:
## build researcher events, student events, and self-study events
## researcher are simply those with NH citations

In [2]:
import sys
import pandas as pd
import time
import datetime

In [3]:
import warnings
warnings.filterwarnings('ignore')
# warnings.filterwarnings(action='once')
warnings.simplefilter(action='ignore', category=FutureWarning)

In [4]:
from nanoHUB.application import Application

application = Application.get_instance()
nanohub_db = application.new_db_engine('nanohub')
nanohub_metrics_db = application.new_db_engine('nanohub_metrics')
wang159_myrmekes_db = application.new_db_engine('wang159_myrmekes')

salesforce = application.new_salesforce_engine()
db_s = salesforce

nanoHUB - Serving Students, Researchers & Instructors
[Error] %s {"error":"invalid_client_id","error_description":"client identifier invalid"}


RuntimeError: No active exception to reraise

## User Lifetime Comparison

In [ ]:
#pull jos_users with the previously found self-study, classroom, and research users
#then just compare the register vs last visit date

In [ ]:
import os
cwd = os.getcwd()

In [ ]:
class_df = pd.read_csv(cwd+'/cached_data/class_ids.csv')
ss_df = pd.read_csv(cwd+'/cached_data/ss_ids.csv')
research_df = pd.read_csv(cwd+'/cached_data/researcher_ids.csv')

In [ ]:
display(class_df.head(2))
display(ss_df.head(2))
display(research_df.head(2))

In [ ]:
sql_query = "select registerDate, lastvisitDate from jos_users where username in " + str(tuple(class_df['username']))

In [ ]:
clustered_dates = pd.read_sql_query(sql_query,nanohub_db)
clustered_dates

In [ ]:
clustered_dates[clustered_dates['lastvisitDate'].isna()].shape[0]

In [ ]:
## determine time diffs
clustered_dates2 = clustered_dates[~clustered_dates['lastvisitDate'].isna()].reset_index()
clustered_dates2['registerDate'] = pd.to_datetime(clustered_dates2['registerDate'])
clustered_dates2['lastvisitDate'] = pd.to_datetime(clustered_dates2['lastvisitDate'])
clustered_dates2['duration'] = clustered_dates2['lastvisitDate'].to_numpy() - clustered_dates2['registerDate'].to_numpy()
display(clustered_dates2.head(2))

In [ ]:
import numpy as np

In [ ]:
clustered_dates2['duration'] = [np.round(np.log10(i.days),2) for i in clustered_dates2['duration'].to_list()]
display(clustered_dates2.head(2))
display(clustered_dates2.tail(2))

In [ ]:
# 50 bins - log days
max_log_interval = 4 #base 10
bins = 50
# 1 day, 2 day, 3 day, 4, 5, 6, 1 week, 2 week, 1 month, 2 month, 1 year, 
bin_vals = [np.round(4/50*(i+1),2) for i in range(bins)] #these are the ending bin vals
bin_vals2 = [np.round(4/50*(i+1),2) for i in range(bins)] #these are the ending bin vals

In [ ]:
clustered_user_vals = []
for i,j in enumerate(bin_vals):
    if i == 0:
#         clustered_user_vals.append(clustered_dates[clustered_dates['lastvisitDate'].isna()].shape[0]\
#                                   +len(np.where(clustered_dates2['duration'] == 1)[0]))
        h_thresh = j
        l_thresh = 0
        clustered_user_vals.append(len(np.where((clustered_dates2['duration'] < h_thresh))[0]))
    else:
        h_thresh = j
        l_thresh = bin_vals[i-1]
        clustered_user_vals.append(len(np.where((clustered_dates2['duration'] > l_thresh) \
                                    & (clustered_dates2['duration'] < h_thresh))[0]))        

        

In [ ]:
clustered_user_vals[0]

In [ ]:
clustered_user_vals2 = []
for i in clustered_user_vals:
    if i != 0:
        clustered_user_vals2.append(np.log10(i))
    else:
        clustered_user_vals2.append(0)

In [ ]:
## import flags and plot

In [ ]:
import re

In [ ]:
## read the dev_flags.txt to determine where to save
with open(cwd+'/dev_flags.txt','r') as f:
    raw_flags = f.read()
flag_pattern = re.compile(r'= \w+')
flags1 = re.findall(flag_pattern,raw_flags)    
flags_result = [i[2:] for i in flags1]
dev_flag = flags_result[0]
prod_flag = flags_result[1]

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
plt.figure()
plt.hist(bin_vals2,bins=bin_vals2,weights=clustered_user_vals,align='left',color='green',edgecolor='white', linewidth=1)
plt.xlabel('User Lifetime (log days)')
plt.xticks(np.arange(0,4.5,0.5))
plt.ylim([0,2500])
plt.ylabel('# of Users (log)')

if dev_flag == 'True':
    plt.savefig(cwd+'/plots_local/simuser_lifetime_class.eps', dpi=1000, bbox_inches='tight')
    plt.savefig(cwd+'/plots_local/simuser_lifetime_class.pdf', dpi=1000, bbox_inches='tight')
    plt.savefig(cwd+'/plots_local/simuser_lifetime_class.png', dpi=1000, bbox_inches='tight')
elif prod_flag == 'True':
    plt.savefig(cwd+'/plots_production/simuser_lifetime_class.eps', dpi=1000, bbox_inches='tight')
    plt.savefig(cwd+'/plots_production/simuser_lifetime_class.pdf', dpi=1000, bbox_inches='tight')
    plt.savefig(cwd+'/plots_production/simuser_lifetime_class.png', dpi=1000, bbox_inches='tight')
else:
    print('no printing flag is active')

In [ ]:
tss_username = ss_df['username'].dropna().to_list()

In [ ]:
sql_query = "select registerDate, lastvisitDate from jos_users where username in " + str(tuple(ss_df['username'].dropna()))

In [ ]:
self_study_dates = pd.read_sql_query(sql_query,nanohub_db)
self_study_dates

In [ ]:
## determine time diffs
self_study_dates2 = self_study_dates[~self_study_dates['lastvisitDate'].isna()].reset_index()
self_study_dates2['registerDate'] = pd.to_datetime(self_study_dates2['registerDate'])
self_study_dates2['lastvisitDate'] = pd.to_datetime(self_study_dates2['lastvisitDate'])
self_study_dates2['duration'] = self_study_dates2['lastvisitDate'].to_numpy() - self_study_dates2['registerDate'].to_numpy()
display(self_study_dates2.head(2))

In [ ]:
self_study_dates2['duration'] = [np.round(np.log10(i.days),2) for i in self_study_dates2['duration'].to_list()]
display(self_study_dates2.head(2))
display(self_study_dates2.tail(2))

In [ ]:
self_user_vals = []
for i,j in enumerate(bin_vals):
    if i == 0:
#         clustered_user_vals.append(clustered_dates[clustered_dates['lastvisitDate'].isna()].shape[0]\
#                                   +len(np.where(clustered_dates2['duration'] == 1)[0]))
        h_thresh = j
        l_thresh = 0
        self_user_vals.append(len(np.where((self_study_dates2['duration'] < h_thresh))[0]))
    else:
        h_thresh = j
        l_thresh = bin_vals[i-1]
        self_user_vals.append(len(np.where((self_study_dates2['duration'] > l_thresh) \
                                    & (self_study_dates2['duration'] < h_thresh))[0]))        

        

In [ ]:
self_user_vals[:5]

In [ ]:
plt.figure()
plt.hist(bin_vals2,bins=bin_vals2,weights=self_user_vals,align='left',color='orange',edgecolor='white', linewidth=1)
plt.xlabel('User Lifetime (log days)')
plt.xticks(np.arange(0,4.5,0.5))
plt.ylim([0,2500])
plt.ylabel('# of Users')

if dev_flag == 'True':
    plt.savefig(cwd+'/plots_local/simuser_lifetime_unclass.eps', dpi=1000, bbox_inches='tight')
    plt.savefig(cwd+'/plots_local/simuser_lifetime_unclass.pdf', dpi=1000, bbox_inches='tight')
    plt.savefig(cwd+'/plots_local/simuser_lifetime_unclass.png', dpi=1000, bbox_inches='tight')
elif prod_flag == 'True':
    plt.savefig(cwd+'/plots_production/simuser_lifetime_unclass.eps', dpi=1000, bbox_inches='tight')
    plt.savefig(cwd+'/plots_production/simuser_lifetime_unclass.pdf', dpi=1000, bbox_inches='tight')
    plt.savefig(cwd+'/plots_production/simuser_lifetime_unclass.png', dpi=1000, bbox_inches='tight')
else:
    print('no printing flag is active')

In [ ]:
sql_query = "select registerDate, lastvisitDate from jos_users where id in " + str(tuple(research_df['nhid']))

In [ ]:
researcher_dates = pd.read_sql_query(sql_query,nanohub_db)
researcher_dates

In [ ]:
## determine time diffs
researcher_dates2 = researcher_dates[~researcher_dates['lastvisitDate'].isna()].reset_index()
researcher_dates2['registerDate'] = pd.to_datetime(researcher_dates2['registerDate'])
researcher_dates2['lastvisitDate'] = pd.to_datetime(researcher_dates2['lastvisitDate'])
researcher_dates2['duration'] = researcher_dates2['lastvisitDate'].to_numpy() - researcher_dates2['registerDate'].to_numpy()
display(researcher_dates2.head(2))

In [ ]:
researcher_dates2['duration'] = [np.round(np.log10(i.days),2) for i in researcher_dates2['duration'].to_list()]
display(researcher_dates2.head(2))
display(researcher_dates2.tail(2))

In [ ]:
researcher_vals = []
for i,j in enumerate(bin_vals):
    if i == 0:
#         clustered_user_vals.append(clustered_dates[clustered_dates['lastvisitDate'].isna()].shape[0]\
#                                   +len(np.where(clustered_dates2['duration'] == 1)[0]))
        h_thresh = j
        l_thresh = 0
        researcher_vals.append(len(np.where((researcher_dates2['duration'] < h_thresh))[0]))
    else:
        h_thresh = j
        l_thresh = bin_vals[i-1]
        researcher_vals.append(len(np.where((researcher_dates2['duration'] > l_thresh) \
                                    & (researcher_dates2['duration'] < h_thresh))[0]))        

        

In [ ]:
researcher_vals[:2]

In [ ]:
plt.figure()
plt.hist(bin_vals2,bins=bin_vals2,weights=researcher_vals,align='left',color='red',edgecolor='white', linewidth=1)
plt.xlabel('User Lifetime (log days)')
plt.xticks(np.arange(0,4.5,0.5))
plt.ylim([0,200])
plt.ylabel('# of Users')

if dev_flag == 'True':
    plt.savefig(cwd+'/plots_local/simuser_lifetime_research.eps', dpi=1000, bbox_inches='tight')
    plt.savefig(cwd+'/plots_local/simuser_lifetime_research.pdf', dpi=1000, bbox_inches='tight')
    plt.savefig(cwd+'/plots_local/simuser_lifetime_research.png', dpi=1000, bbox_inches='tight')
elif prod_flag == 'True':
    plt.savefig(cwd+'/plots_production/simuser_lifetime_research.eps', dpi=1000, bbox_inches='tight')
    plt.savefig(cwd+'/plots_production/simuser_lifetime_research.pdf', dpi=1000, bbox_inches='tight')
    plt.savefig(cwd+'/plots_production/simuser_lifetime_research.png', dpi=1000, bbox_inches='tight')
else:
    print('no printing flag is active')